# Сохранение и возобновление обучения в Tensorflow

До этого момента мы с вами работали с "игрушечными" данными и основной целью было понять основные механизмы работы фреймворка, обучения и построения моделей.

Раньше мы не думали о том, как можно использовать обученную модель в будущем и сохранять параметры, которые мы получили во время обучения.

Именно об этих возможностях, обязательных в практическом применении сетей, и пойдет речь в этом уроке.

## Датасет

В этом уроке мы будем работать с датасетом Pima Indian Diabetes, в котором приведены данные о состоянии здоровья людей. Помимо физиологических показаний, в данных есть информация о наличии у человека диабета. Это и будет считаться классом.

In [1]:
# загрузим данные
! wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
import pandas as pd

dataset = pd.read_csv("pima-indians-diabetes.data.csv", header=None)
dataset.columns = ["number_times_pregnant",
"plasma_glucose_concent","blood_pressure",
"skin_thickness", "insulin", "BMI", "diabetes_pedigree", "age", "has_diabetes"]
dataset.head()

--2024-05-02 16:51:06--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.data.csv.1’

pima-indians-diabet 100%[===================>]  22,73K  --.-KB/s    in 0,008s  

2024-05-02 16:51:07 (2,69 MB/s) - ‘pima-indians-diabetes.data.csv.1’ saved [23278/23278]



,number_times_pregnant,plasma_glucose_concent,blood_pressure,skin_thickness,insulin,BMI,diabetes_pedigree,age,has_diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(10)
X = dataset.drop(['has_diabetes'], axis=1).values
y = dataset['has_diabetes'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}, Labels ratio: {y_train.mean():.2f}")

Train shape: (614, 8), Test shape: (154, 8), Labels ratio: 0.34


## Сохранение и загрузка моделей

Т.к. параметры нашей модели трансформируются путем оптимизации из случайных -- в полезные для нашей задачи, нам бы хотелось сохранять этот прогресс. Было бы неразумно каждый раз обучать модель заново. Сохранение параметров модели во время обучения называют -- чекпоинтом (checkpoint).

Подходы к сохранению моделей могут варьироваться в зависимости от количества данных и сложности модели. Все они сводятся к правилам по которым решается когда сохранять модель. Приведем самые популярные из таких правил:
*   Сохранение в конце каждой эпохи
*   Сохранение каждые k эпох
*   Сохранение только последней эпохи
*   Сохранение весов при улучшении метрики


In [3]:
import tensorflow as tf
import numpy as np
#tf.enable_eager_execution()
print(tf.__version__)

2024-05-02 16:51:08.034329: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 16:51:08.997283: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.16.1


Для начала обучим модель и убедимся, что все работает:

In [43]:
def get_compiled_model():
    """
    Функция возвращает скомпилированную модель для бинарной классификации
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(24, input_dim=8, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=1)

Epoch 1/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5157 - loss: 4.7203 - val_accuracy: 0.5844 - val_loss: 1.0106
Epoch 2/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6020 - loss: 0.9293 - val_accuracy: 0.5974 - val_loss: 0.8199
Epoch 3/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6444 - loss: 0.7165 - val_accuracy: 0.6169 - val_loss: 0.6668
Epoch 4/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6982 - loss: 0.5951 - val_accuracy: 0.6299 - val_loss: 0.6471
Epoch 5/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7044 - loss: 0.5774 - val_accuracy: 0.6753 - val_loss: 0.6296
Epoch 6/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6822 - loss: 0.6056 - val_accuracy: 0.6104 - val_loss: 0.6370
Epoch 7/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7242 - loss: 0.5735 - val_accuracy: 0.6558 - val_loss: 0.6603
Epoch 8/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6796 - loss: 0.6288 - val_accuracy: 0.6558 - 

### Сохранение модели

За запись чекпоинтов в keras отвечает **tf.keras.callbacks.ModelCheckpoint**. Как видно из пути к модулю -- он относится к коллбекам (callbacks). Можно смотреть на них как на функции, которые вызывает метод model.fit внутри себя.

То есть каждый раз, когда отработает одна из эпох цикла, будет запускаться функция, которую мы укажем. Бывают коллбеки, которые вызываются на каждой итерации (например [прогресс бар](https://keras.io/callbacks/#progbarlogger), который мы видим при обучении -- это тоже коллбэк). Чуть ниже мы разберем конкретный пример.

Вернемся к **tf.keras.callbacks.ModelCheckpoint**.

```
tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', ...)
```

Он имеет один обязательный параметр -- имя чекпоинта. Оно может быть отформатировано с использованием служебных переменных (пример ниже). Из необязательных, но полезных параметров:

*   monitor (str): название метрики за которой нужно "следить"
*   verbose -- печатать ли дополнительную информацию
*   save_best_only (bool): сохранять чекпоинт только после улучшения метрики, указанной в monitor
*   save_weights_only (bool): помимо весов модели можно еще сохранять ее архитектуру и состояние оптимизатора. В примерах мы так и будем делать, если не сказано обратное. По умолчанию -- false.
*   mode (str): если метрику нужно максимизировать -- max, иначе -- min.


**Таким образом, если выбрано save_best_only, то чекпоинты будут сохранятся только после тех эпох, когда произошло улучшение метрики, которую мы указали в monitor. Т.к. понятие "лучше" -- разное для разных метрик, необходимо также указать режим -- max/min (больше -- лучше/меньше -- лучше). Пример таких метрик -- лосс (min) и точность (max)**.

In [44]:
from pathlib import Path
path = Path("model_1")
path.mkdir(exist_ok=True) # создаем папку на диске
cpt_filename = "{epoch:02d}_checkpoint_{val_accuracy:.2f}.keras"
cpt_path = str(path / cpt_filename)

checkpoint = tf.keras.callbacks.ModelCheckpoint(cpt_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [37]:
tf.keras.callbacks.ModelCheckpoint?

Init signature:
tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
    initial_value_threshold=None,
)
Docstring:     
Callback to save the Keras model or model weights at some frequency.

`ModelCheckpoint` callback is used in conjunction with training using
`model.fit()` to save a model or weights (in a checkpoint file) at some
interval, so the model or weights can be loaded later to continue the
training from the state saved.

A few options this callback provides include:

- Whether to only keep the model that has achieved the "best performance" so
  far, or whether to save the model at the end of every epoch regardless of
  performance.
- Definition of "best"; which quantity to monitor and whether it should be
  maximized or minimized.
- The frequency it should save at. Currently, the callback supports saving
  at the end of every epoch, or after a fix

In [31]:
"a={a_val}, b={b_val}".format(a_val=2, b_val=34)

'a=2, b=34'

In [38]:
cpt_path

'model_1/{epoch:02d}_checkpoint_{val_accuracy:.2f}.keras'

In [39]:
# внутри цикла имя чекпоинта будет сгенерировано примерно таким образом:
for epoch in range(4):
    print(cpt_path.format(epoch=epoch, val_accuracy=np.random.rand()))

model_1/00_checkpoint_0.97.keras
model_1/01_checkpoint_0.99.keras
model_1/02_checkpoint_0.78.keras
model_1/03_checkpoint_0.94.keras


Мы используем hdf5 -- основной формат для keras. Для Tensorflow -- это ckpt. Сохранять в этом расширении можно аналогичным образом. Подробнее можно прочитать [здесь](https://www.tensorflow.org/tutorials/keras/save_and_restore_models).

In [45]:
model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=0,
          callbacks=[checkpoint])


Epoch 1: val_accuracy improved from -inf to 0.62338, saving model to model_1/01_checkpoint_0.62.keras

Epoch 2: val_accuracy improved from 0.62338 to 0.66234, saving model to model_1/02_checkpoint_0.66.keras

Epoch 3: val_accuracy did not improve from 0.66234

Epoch 4: val_accuracy did not improve from 0.66234

Epoch 5: val_accuracy did not improve from 0.66234

Epoch 6: val_accuracy improved from 0.66234 to 0.67532, saving model to model_1/06_checkpoint_0.68.keras

Epoch 7: val_accuracy did not improve from 0.67532

Epoch 8: val_accuracy improved from 0.67532 to 0.68182, saving model to model_1/08_checkpoint_0.68.keras

Epoch 9: val_accuracy did not improve from 0.68182

Epoch 10: val_accuracy did not improve from 0.68182

Epoch 11: val_accuracy did not improve from 0.68182

Epoch 12: val_accuracy did not improve from 0.68182

Epoch 13: val_accuracy did not improve from 0.68182

Epoch 14: val_accuracy did not improve from 0.68182

Epoch 15: val_accuracy did not improve from 0.68182



Убедимся ниже, что файлы сохранены:

In [46]:
!ls "model_1/"

01_checkpoint_0.62.keras  101_checkpoint_0.75.keras  23_checkpoint_0.71.keras
02_checkpoint_0.66.keras  107_checkpoint_0.75.keras  30_checkpoint_0.72.keras
06_checkpoint_0.68.keras  124_checkpoint_0.76.keras  62_checkpoint_0.73.keras
08_checkpoint_0.68.keras  140_checkpoint_0.79.keras  70_checkpoint_0.73.keras


### Загрузка модели

Загрузить модель можно двумя способами. Определить модель и загрузить в нее веса, с использованием *model.load_weights()*. Или tf.keras.models.load_model для автоматического создания модели и загрузки весов (мы можем так сделать т.к. при создании чекпоинтов выставили save_weights_only=False, т.е. сохранили и описание модели).

In [15]:
model = get_compiled_model() # определим случайно инициализированную модель
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of random model {acc*100 :.2f}%")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3560 - loss: 23.4429  
Accuracy of random model 38.31%


In [47]:
model.load_weights("model_1/70_checkpoint_0.73.keras") # загружаем веса. обратите внимание, модель уже создана!
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - accuracy: 0.7229 - loss: 0.5672
Accuracy of restored model 73.38%


In [48]:
# т.к. мы сохраняли и веса и модель, то мы можем загрузить модель не определяя ее
model = tf.keras.models.load_model("model_1/70_checkpoint_0.73.keras")
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7229 - loss: 0.5672  
Accuracy of restored model 73.38%


### Сохранение модели в один чекпоинт
Если не форматировать название файла, то все чекпоинты будут записаны в него:

In [49]:
path = Path("model_2")
path.mkdir(exist_ok=True)
cpt_filename = "checkpoint_best.keras"
cpt_path =str(path / cpt_filename)

checkpoint = tf.keras.callbacks.ModelCheckpoint(cpt_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=0,
          callbacks=[checkpoint])


Epoch 1: val_accuracy improved from -inf to 0.61688, saving model to model_2/checkpoint_best.keras

Epoch 2: val_accuracy did not improve from 0.61688

Epoch 3: val_accuracy improved from 0.61688 to 0.62338, saving model to model_2/checkpoint_best.keras

Epoch 4: val_accuracy improved from 0.62338 to 0.64286, saving model to model_2/checkpoint_best.keras

Epoch 5: val_accuracy did not improve from 0.64286

Epoch 6: val_accuracy improved from 0.64286 to 0.64935, saving model to model_2/checkpoint_best.keras

Epoch 7: val_accuracy did not improve from 0.64935

Epoch 8: val_accuracy did not improve from 0.64935

Epoch 9: val_accuracy improved from 0.64935 to 0.67532, saving model to model_2/checkpoint_best.keras

Epoch 10: val_accuracy did not improve from 0.67532

Epoch 11: val_accuracy did not improve from 0.67532

Epoch 12: val_accuracy did not improve from 0.67532

Epoch 13: val_accuracy did not improve from 0.67532

Epoch 14: val_accuracy did not improve from 0.67532

Epoch 15: val_

In [50]:
! ls model_2

checkpoint_best.keras


In [51]:
model = tf.keras.models.load_model("model_2/checkpoint_best.keras")
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7689 - loss: 0.6175  
Accuracy of restored model 75.97%


Сохранять один файл или несколько -- решать вам, это зависит от задач и принятых практик ведения экспериментов, например, в каких-то случаях может пригодится сравнить предсказания нескольких последних моделей.


### Продолжение процесса обучения

In [52]:
# мы можем продолжить процесс обучения, обратите внимание на initial_epoch
model = tf.keras.models.load_model("model_2/checkpoint_best.keras")
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=300, batch_size=64, verbose=0,
          callbacks=[checkpoint], initial_epoch=150)


Epoch 151: val_accuracy did not improve from 0.75974

Epoch 152: val_accuracy did not improve from 0.75974

Epoch 153: val_accuracy did not improve from 0.75974

Epoch 154: val_accuracy did not improve from 0.75974

Epoch 155: val_accuracy did not improve from 0.75974

Epoch 156: val_accuracy did not improve from 0.75974

Epoch 157: val_accuracy did not improve from 0.75974

Epoch 158: val_accuracy did not improve from 0.75974

Epoch 159: val_accuracy did not improve from 0.75974

Epoch 160: val_accuracy did not improve from 0.75974

Epoch 161: val_accuracy did not improve from 0.75974

Epoch 162: val_accuracy did not improve from 0.75974

Epoch 163: val_accuracy did not improve from 0.75974

Epoch 164: val_accuracy did not improve from 0.75974

Epoch 165: val_accuracy did not improve from 0.75974

Epoch 166: val_accuracy did not improve from 0.75974

Epoch 167: val_accuracy did not improve from 0.75974

Epoch 168: val_accuracy did not improve from 0.75974

Epoch 169: val_accuracy did

## Заключение

В этом уроке мы узнали как сохранять прогресс обучения и использовать полученные модели.

Следующий этап -- [домашнее задание](https://colab.research.google.com/drive/1G6iHt4PUVrjSHQ80wnCYm3gOtx_Gl1gE), в котором вы примените все полученные знания из этого модуля. Вы снова вернетесь к реализации классификатора известных вам рукописных цифр, но в этот раз сделаете это с помощью Tensorflow.